In [11]:
import pandas as pd
import altair as alt
alt.renderers.set_embed_options(theme='dark')


RendererRegistry.enable('default')

In [12]:
def claim(claim_hash, cols_claim):
    try:
        df_claim = pd.read_json(
            f"https://api.flipsidecrypto.com/api/v2/queries/{claim_hash}/data/latest",
            convert_dates=["BLOCK_TIMESTAMP"],
        )
    except:
        return pd.DataFrame(columns = cols_claim[claim_hash])
    if(len(df_claim.columns)==0):
        return pd.DataFrame(columns = cols_claim[claim_hash])
    return df_claim

In [13]:
p2_users = '07f87a6f-5bc6-49f3-a1bb-8e7d545ba95a'
p2_hourly = 'de5232e1-5d57-4232-900d-1d09d2c9438e'

In [14]:
cols_claim = {
    p2_users: ["DEPOSIT_AMOUNT","HR","NET_DEPOSITED_AMOUNT","SENDER","WITHDRAWN_AMOUNT","WITHDRAWN_AMOUNT_PHASE2"],
    p2_hourly: ['HR', 'WITH_AMOUNT', 'WITH_TX', 'WITH_USERS']
}

In [63]:
user_stats = '499224b4-30a6-43d7-80b9-3a019cbb1d3d'
user_stats_df = claim(user_stats, cols_claim)
user_stats_df.to_csv('../data/user_deposit_p1_s1.csv')

In [70]:
user_stats_df = pd.read_csv('https://raw.githubusercontent.com/IncioMan/prism_forge/master/data/user_deposit_p1_s1.csv',index_col=0)
user_stats_df.columns = [c.lower() for c in user_stats_df.columns]
user_stats_df

,deposit_amount,deposit_txs,sender,withdrawn_amount,withdraw_txs
0,100.000000,1,terra1kne4ul6n405w769k2p6swx7x4ep30rv24099r8,0.0,0
1,10000.000000,1,terra1nf4spkar3cdc9x5cflvxretr4qug43tqv58qxl,0.0,0
2,14970.608264,1,terra13ahpu2w8lrl7z7g6tu78un67w8vdlvh5ghkd9w,0.0,0
3,5000.000000,1,terra1cwgaynf2vtgrjvkj4xhya0eetytz5p0t8rhchu,0.0,0
4,1000.000000,1,terra1jnvga8p33vzgsjq05y6xmrpxk9ernl5lgsrnx9,0.0,0
...,...,...,...,...,...
3584,300.000000,1,terra1vmdzwhmxctyw3jvq688gek4jj27wlyz4da7uzp,0.0,0
3585,6300.000000,3,terra1qhxpsds2tlr2wrq864w40qutd9f9puz9gukrq5,0.0,0
3586,100.000000,1,terra1gan8naj2rvg5n0w4e54835nfuv7w0xtkwnmy4d,0.0,0
3587,1000.000000,1,terra13sz69jkkl7p8vqdxps2af5lqqj5cae9dkexmel,1000.0,1


In [71]:
with_phase1 = pd.read_csv('https://raw.githubusercontent.com/IncioMan/prism_forge/master/data/with_phase1.csv', index_col=0)
with_phase1.deposit=with_phase1.deposit/1000000
with_phase1.total_deposit=with_phase1.total_deposit/1000000
with_phase1.withdrawable_amount=with_phase1.withdrawable_amount/1000000
with_phase1.tokens_to_claim=with_phase1.tokens_to_claim/1000000
with_phase1

,sender,deposit,total_deposit,withdrawable_amount,tokens_to_claim,can_claim
0,terra1v6ujue0uqd9nzgtqvuhwncaqd9q6hacvsfur7x,400.0,9.203460e+06,400.0,3042.334072,False
1,terra17js7wjaq8w80khztfa9ydz0etdgzy4l7t8yfsa,100.0,9.203460e+06,100.0,760.583518,False
2,terra1frsd6nayz25encp6w6m8z00g5ue4j6fp4q0pza,0.0,9.203460e+06,0.0,0.000000,False
3,terra1lcrwyxn8apvxu9jdvu6jk6myeey0z64e22du4k,185.0,9.203460e+06,185.0,1407.079508,False
4,terra1nrg8s0wtd2z0xyl26u669qh87qcn2ql0wepvhj,0.0,9.203460e+06,0.0,0.000000,False
5,terra1hx6ea4yxz7lhex5xv329475zlrzxvx57qmjylm,0.0,9.203460e+06,0.0,0.000000,False
6,terra1t4qagk0vukvwttjm3myzdqav8yr2gx6d7jssra,0.0,9.203460e+06,0.0,0.000000,False
7,terra1xswthhdas6d5xu4x2ffa3zs2cc7mh80ywfavg4,155.0,9.203460e+06,155.0,1178.904452,False
8,terra1f8qwvzrqt0plehvck3gst2clvsvxccckjk37hz,1000.0,9.203460e+06,1000.0,7605.835180,False
9,terra1q62uql0ht63x9ax7pwx35wpthrufltm42evrhf,115.0,9.203460e+06,115.0,874.671045,False


In [72]:
p2_users_df = claim(p2_users, cols_claim)
p2_hourly_df = claim(p2_hourly, cols_claim)

In [73]:
p2_users_df.columns = [c.lower() for c in p2_users_df.columns]
p2_users_df

,deposit_amount,hr,net_deposited_amount,sender,withdrawn_amount,withdrawn_amount_phase2
0,1000.0,2022-02-02 09:00:00.000,400.0,terra1v6ujue0uqd9nzgtqvuhwncaqd9q6hacvsfur7x,600.0,600.0
1,101.0,2022-02-01 11:00:00.000,100.0,terra17js7wjaq8w80khztfa9ydz0etdgzy4l7t8yfsa,1.0,1.0
2,99500.0,2022-02-01 20:00:00.000,0.0,terra1frsd6nayz25encp6w6m8z00g5ue4j6fp4q0pza,99500.0,99500.0
3,185.0,2022-02-01 12:00:00.000,35.0,terra1lcrwyxn8apvxu9jdvu6jk6myeey0z64e22du4k,150.0,150.0
4,100.0,2022-02-01 22:00:00.000,0.0,terra1nrg8s0wtd2z0xyl26u669qh87qcn2ql0wepvhj,100.0,100.0
...,...,...,...,...,...,...
1879,555.0,None,555.0,terra1qg7axjshlpdsh0t5rw07trp9haytqvtszwgrcx,0.0,NaN
1880,500.0,None,500.0,terra1md40c76suxrakhc0453n2lyc0xw3xeezy0mpmd,0.0,NaN
1881,200.0,None,200.0,terra1ka73l5hfmqw3xsf6lgmn8rts2j03ekfv923qna,0.0,NaN
1882,1000.0,None,1000.0,terra15kewrs7gxsp3m569ufmf3ay23s3euln8ec3e7d,0.0,NaN


In [100]:
users_with = p2_users_df.merge(with_phase1,on='sender')[['sender','net_deposited_amount','deposit','withdrawable_amount']]
users_with=users_with.rename(columns={'net_deposited_amount':'deposited_p1'})
users_with['WITHDRAWN_AMOUNT_PHASE2'] = users_with.deposited_p1-users_with.deposit
users_with['has_withdrawn_p2'] = users_with.deposit != users_with.deposited_p1
users_with

,sender,deposited_p1,deposit,withdrawable_amount,WITHDRAWN_AMOUNT_PHASE2,has_withdrawn_p2
0,terra1v6ujue0uqd9nzgtqvuhwncaqd9q6hacvsfur7x,400.0,400.0,400.0,0.0,False
1,terra17js7wjaq8w80khztfa9ydz0etdgzy4l7t8yfsa,100.0,100.0,100.0,0.0,False
2,terra1frsd6nayz25encp6w6m8z00g5ue4j6fp4q0pza,0.0,0.0,0.0,0.0,False
3,terra1lcrwyxn8apvxu9jdvu6jk6myeey0z64e22du4k,35.0,185.0,185.0,-150.0,True
4,terra1nrg8s0wtd2z0xyl26u669qh87qcn2ql0wepvhj,0.0,0.0,0.0,0.0,False
5,terra1hx6ea4yxz7lhex5xv329475zlrzxvx57qmjylm,0.0,0.0,0.0,0.0,False
6,terra1t4qagk0vukvwttjm3myzdqav8yr2gx6d7jssra,0.0,0.0,0.0,0.0,False
7,terra1xswthhdas6d5xu4x2ffa3zs2cc7mh80ywfavg4,155.0,155.0,155.0,0.0,False
8,terra1f8qwvzrqt0plehvck3gst2clvsvxccckjk37hz,1000.0,1000.0,1000.0,0.0,False
9,terra1q62uql0ht63x9ax7pwx35wpthrufltm42evrhf,115.0,115.0,115.0,0.0,False


In [75]:
cols_dict = {
    "DEPOSIT_AMOUNT":"Amount deposited",
    "DEPOSIT_TXS":"N° of deposit txs",
    "DEP_USERS":"N° of depositing users",
    "HR":"Time",
    "NET_AMOUNT":"Net amount deposited",
    "TOT_TXS":"N° of txs",
    "TOT_USERS":"N° of users",
    "WITH_AMOUNT":"Withdrawn amount",
    "WITH_TXS":"N° of withdraw txs",
    "WITH_USERS":"N° of withdrawing users",
    'AMOUNT':'Amount deposited (UST)',
    'AVG_BALANCE_USD':'Average balance in the last 30 days (UST)',
    'N_TXS':'N° of txs',
    'SENDER':'User address',
    'N_USERS':'N° of users',
    'BUCKET':'Deposit range',
    'bucket_name':'Deposit bucket',
    'cumsum_ust':'Cumulative UST amount',
    'MIN_DATE':'Date wallet creation',
    'ADDRESS_COUNT':'Number of wallets',
    'PERC_WITHDRAWN':'% withdrawn of the deposited amount',
    'net_ust': 'Amount UST',
    'perc_withdrawn_cat':'Percentage withdrawn UST bucket',
    'DEP_CAT':'Deposited UST range'
}

In [101]:
#Total withdrawn phase 2 UST
tot_with_ust = users_with['WITHDRAWN_AMOUNT_PHASE2'].sum()
tot_with_ust

-150.0

In [102]:
#Total net dep phase 1 UST
tot_ust_p1 = users_with['deposited_p1'].sum()
tot_ust_p1

1805.0

In [103]:
#Total left UST
tot_net_ust = users_with.deposit.sum()
tot_net_ust

1955.0

In [104]:
#% withdrawn
perc_with_p2 = tot_with_ust/tot_ust_p1 * 100
perc_with_p2

-8.310249307479225

In [106]:
#current price
curr_price = tot_net_ust/70000000
curr_price

2.7928571428571427e-05

In [107]:
#% users with
p_users_with_p2 = (users_with.has_withdrawn_p2.sum()/len(p2_users_df))*100
p_users_with_p2

0.05307855626326964

In [108]:
#Floor price
left_to_with = users_with.withdrawable_amount.sum()

In [109]:
p2_users_df.head()

,deposit_amount,hr,net_deposited_amount,sender,withdrawn_amount,withdrawn_amount_phase2,max_with_hour
0,1000.0,2022-02-02 09:00:00.000,400.0,terra1v6ujue0uqd9nzgtqvuhwncaqd9q6hacvsfur7x,600.0,600.0,2022-02-03 10:00:00.000
1,101.0,2022-02-01 11:00:00.000,100.0,terra17js7wjaq8w80khztfa9ydz0etdgzy4l7t8yfsa,1.0,1.0,2022-02-03 10:00:00.000
2,99500.0,2022-02-01 20:00:00.000,0.0,terra1frsd6nayz25encp6w6m8z00g5ue4j6fp4q0pza,99500.0,99500.0,2022-02-03 10:00:00.000
3,185.0,2022-02-01 12:00:00.000,35.0,terra1lcrwyxn8apvxu9jdvu6jk6myeey0z64e22du4k,150.0,150.0,2022-02-03 10:00:00.000
4,100.0,2022-02-01 22:00:00.000,0.0,terra1nrg8s0wtd2z0xyl26u669qh87qcn2ql0wepvhj,100.0,100.0,2022-02-03 10:00:00.000


In [110]:
df = pd.DataFrame([[tot_ust_p1-tot_with_ust,tot_with_ust],['Still deposited','Withdrawn']]).T
df.columns = ['UST','Type']
df

,UST,Type
0,1955.0,Still deposited
1,-150.0,Withdrawn


In [111]:
alt.Chart(df).mark_arc().encode(
    theta=alt.Theta(field="UST", type="quantitative"),
    color=alt.Color(field="Type", type="nominal",
            scale=alt.Scale(scheme='pastel1'),
            legend=alt.Legend(
            orient='none',
            padding=10,
            legendY=-10,
            direction='vertical'))
).configure_view(strokeOpacity=0)

alt.Chart(...)

In [113]:
#UST left to be withdrawn
users_with.withdrawable_amount.sum()

1955.0

In [114]:
#Floor price
floor_price = (tot_net_ust - left_to_with)/70000000
floor_price

0.0

In [115]:
p2_hourly_df['HR'] = '2022/' + p2_hourly_df['HR'] + ':00'
p2_hourly_df.head()

,HR,WITH_AMOUNT,WITH_TX,WITH_USERS
0,2022/02/02 11:00:00,1070.000000,2,2
1,2022/02/02 09:00:00,1150.000000,4,4
2,2022/02/01 20:00:00,104711.479309,5,5
3,2022/02/01 12:00:00,857.670794,3,3
4,2022/02/01 10:00:00,19312.000000,3,3


In [116]:
alt.Chart(p2_hourly_df.rename(columns=cols_dict)).mark_bar().encode(
    x=cols_dict['HR']+':T',
    y=cols_dict['WITH_AMOUNT']+":Q",
    tooltip=[cols_dict['HR']+':T',cols_dict['WITH_AMOUNT']+":Q"]
).configure_mark(
    color='#fab0ba'
).properties(width=700).configure_axisX(
    labelAngle=30
).configure_view(strokeOpacity=0)

alt.Chart(...)

In [117]:
alt.Chart(p2_hourly_df.rename(columns=cols_dict)).mark_bar().encode(
    x=cols_dict['HR']+':T',
    y=cols_dict['WITH_USERS']+":Q",
    tooltip=[cols_dict['HR']+':T',cols_dict['WITH_USERS']+":Q"]
).configure_mark(
    color='#fab0ba'
).properties(width=700).configure_axisX(
    labelAngle=30
).configure_view(strokeOpacity=0)

alt.Chart(...)

In [141]:
users_with.WITHDRAWN_AMOUNT_PHASE2 = users_with.WITHDRAWN_AMOUNT_PHASE2.apply(lambda x: x if x >=0 else 0)

In [142]:
with_users_df = users_with[users_with.has_withdrawn_p2&users_with['deposited_p1']>0]
with_users_df['perc_withdrawn'] = with_users_df[with_users_df.has_withdrawn_p2]['WITHDRAWN_AMOUNT_PHASE2']/with_users_df['deposited_p1']*100
with_users_df['perc_withdrawn_cat'] = (with_users_df['perc_withdrawn']/100).apply(lambda x: int(x))
df2 = with_users_df.groupby('perc_withdrawn_cat').sender.count()

C:\Users\aincerti\AppData\Local\Temp\ipykernel_18152\3153063284.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  with_users_df['perc_withdrawn'] = with_users_df[with_users_df.has_withdrawn_p2]['WITHDRAWN_AMOUNT_PHASE2']/with_users_df['deposited_p1']*100
C:\Users\aincerti\AppData\Local\Temp\ipykernel_18152\3153063284.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  with_users_df['perc_withdrawn_cat'] = (with_users_df['perc_withdrawn']/100).apply(lambda x: int(x))


In [144]:
perc_cat = list(range(10))
cat = pd.DataFrame([0]*10,perc_cat)
df3 = cat.join(df2,how='outer')
df3

,0,sender
0,0,1.0
1,0,NaN
2,0,NaN
3,0,NaN
4,0,NaN
5,0,NaN
6,0,NaN
7,0,NaN
8,0,NaN
9,0,NaN


In [148]:
perc_cat = list(range(10))
cat = pd.DataFrame([0]*10,perc_cat)
df3 = cat.join(df2,how='outer')
df3.index =  ['0%-10%','10%-20%','20%-30%','30%-40%','40%-50%','50%-60%','60%-70%','70%-80%','80%-90%','90%-100%']
df3 = df3.sender.fillna(0).reset_index()
with_perc_buckets=df3.rename(columns={'index':'PERC_WITHDRAWN','sender':'TOT_USERS'})

In [149]:
with_perc_buckets

,PERC_WITHDRAWN,TOT_USERS
0,0%-10%,1.0
1,10%-20%,0.0
2,20%-30%,0.0
3,30%-40%,0.0
4,40%-50%,0.0
5,50%-60%,0.0
6,60%-70%,0.0
7,70%-80%,0.0
8,80%-90%,0.0
9,90%-100%,0.0


In [150]:
alt.Chart(with_perc_buckets.sort_values(by='PERC_WITHDRAWN').rename(columns=cols_dict)).mark_bar().encode(
    y=alt.X(cols_dict['PERC_WITHDRAWN']+":N", sort=alt.EncodingSortField(order='ascending')),
    x=cols_dict['TOT_USERS']+":Q",
    tooltip=[cols_dict['PERC_WITHDRAWN']+":N",cols_dict['TOT_USERS']+":Q"]
).configure_mark(
    color='#F1705F'
).properties(height=300).configure_axisX(
    labelAngle=0
).configure_view(strokeOpacity=0)

alt.Chart(...)

In [151]:
p2_hourly_df = p2_hourly_df.sort_values(by='HR')
p2_hourly_df['cumsum_with'] = p2_hourly_df.sort_values(by='HR').WITH_AMOUNT.cumsum()
p2_hourly_df['net_ust'] = tot_ust_p1 - p2_hourly_df['cumsum_with']
p2_hourly_df.head()

,HR,WITH_AMOUNT,WITH_TX,WITH_USERS,cumsum_with,net_ust
22,2022/02/01 07:00:00,501183.642552,11,11,501183.642552,-499378.642552
14,2022/02/01 08:00:00,5923.464239,3,3,507107.106791,-505302.106791
27,2022/02/01 09:00:00,1250.000000,2,2,508357.106791,-506552.106791
4,2022/02/01 10:00:00,19312.000000,3,3,527669.106791,-525864.106791
11,2022/02/01 11:00:00,24691.000000,7,7,552360.106791,-550555.106791


In [152]:
alt.Chart(p2_hourly_df.rename(columns=cols_dict)).mark_line(point=True).encode(
    x=alt.X(cols_dict['HR']+':T', sort=alt.EncodingSortField(order='ascending')),
    y=cols_dict['net_ust']+":Q",
    tooltip=[alt.Tooltip(cols_dict['HR']+':T', format='%Y-%m-%d %H:%M'), alt.Tooltip(cols_dict['net_ust']+":Q")]
).configure_mark(
    color='#F1705F'
).properties(width=700).configure_axisX(
).configure_view(strokeOpacity=0)

alt.Chart(...)

In [141]:
with_users_df['DEP_CAT'] = (with_users_df['NET_DEPOSITED_AMOUNT']/1000).apply(int)
df = with_users_df.groupby(['DEP_CAT','perc_withdrawn_cat']).SENDER.count()
df = df.reset_index()
df

C:\Users\aincerti\AppData\Local\Temp\ipykernel_16196\875801285.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  with_users_df['DEP_CAT'] = (with_users_df['NET_DEPOSITED_AMOUNT']/1000).apply(int)


,DEP_CAT,perc_withdrawn_cat,SENDER
0,0,0,9
1,0,1,5
2,0,2,1
3,0,3,2
4,0,4,2
5,0,5,1
6,0,8,1
7,1,0,6
8,1,1,2
9,2,0,1


In [142]:
alt.Chart(df).mark_rect().encode(
    x=alt.X('perc_withdrawn_cat:O', sort=alt.EncodingSortField(order='ascending')),
    y=alt.Y('DEP_CAT:O', sort=alt.EncodingSortField(order='descending')),
    color='SENDER:Q'
)

alt.Chart(...)

In [143]:
df

,DEP_CAT,perc_withdrawn_cat,SENDER
0,0,0,9
1,0,1,5
2,0,2,1
3,0,3,2
4,0,4,2
5,0,5,1
6,0,8,1
7,1,0,6
8,1,1,2
9,2,0,1


In [144]:
import random
df1 = pd.DataFrame([list(range(0,100,5)),
                    ["0%-5%",	"5%-10%",	"10%-15%",	"15%-20%",	"20%-25%",	"25%-30%",	"30%-35%",	
"35%-40%",	"40%-45%",	"45%-50%",	"50%-55%",	"55%-60%",	"60%-65%",	
"65%-70%",	"70%-75%",	"75%-80%",	"80%-85%",	"85%-90%",	"90%-95%",	"95%-100%"]]).T
df2 = pd.DataFrame([list(range(0,651,50)),
                    ["0-50k", "50k-100k",	"100k-150k","150k-200k",	"200k-250k",	"250k-300k",	"300k-350k",
	"350k-400k",	"400k-450k",	"450k-500k",	"500k-550k",	"550k-600k",	"600k-650k", 	"650k-700k"]]).T
heatmap_val = df2.merge(df1, how='cross')
heatmap_val.columns = ['DEP_CAT','DEP_CAT_label','perc_withdrawn_cat','perc_withdrawn_cat_label']
df = heatmap_val.merge(df, on=['perc_withdrawn_cat', 'DEP_CAT'], how='left').fillna(0)
df['SENDER'] = df['SENDER'].apply(lambda v: random.random() * 1000)
heatmap_data_df = df.sort_values(by=['perc_withdrawn_cat', 'DEP_CAT'], ascending=[True,False])
heatmap_data_df

,DEP_CAT,DEP_CAT_label,perc_withdrawn_cat,perc_withdrawn_cat_label,SENDER
260,650,650k-700k,0,0%-5%,108.053645
240,600,600k-650k,0,0%-5%,692.077292
220,550,550k-600k,0,0%-5%,672.644879
200,500,500k-550k,0,0%-5%,718.095257
180,450,450k-500k,0,0%-5%,318.125150
...,...,...,...,...,...
99,200,200k-250k,95,95%-100%,194.608781
79,150,150k-200k,95,95%-100%,339.935606
59,100,100k-150k,95,95%-100%,336.824238
39,50,50k-100k,95,95%-100%,969.234680


In [145]:
alt.Chart((heatmap_data_df.rename(columns=cols_dict))).mark_rect().encode(
    x=alt.X(cols_dict['perc_withdrawn_cat']+':O', sort=alt.EncodingSortField(order='ascending')),
    y=alt.Y(cols_dict['DEP_CAT']+':O', sort=alt.EncodingSortField(order='ascending')),
    color=alt.Color(cols_dict['SENDER']+':Q',
            scale=alt.Scale(scheme='redpurple')),
    tooltip=['DEP_CAT_label'+':N','perc_withdrawn_cat_label'+':N']
)

alt.Chart(...)